## Выявление закономерностей на основе данных МФЦ

Гражданин обращается в МФЦ за получением различных услуг в разное время. На каждое обращение заявителя (прием заявления и документов, консультация) затрачивается определенное время как работником МФЦ, так и заявителем. Однако, эти услуги, вероятно, можно получить за одно обращение и сэкономить время, если выявить те услуги, за которыми обращаются разные граждане в одинаковой (определенной) последовательности. Необходимо предсказать за какой следующей услугой обратится гражданин. В задаче используется Accuracy без модификаций.

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split,KFold
from catboost import CatBoostClassifier, Pool
import lightgbm as lgb

import gc
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_column', 10000)
pd.options.mode.use_inf_as_na = True
display(HTML("<style>.container { width:80% !important; }</style>"))

### Загрузка датасета

In [3]:
%%time
data = pd.read_csv('train.csv',header=None)
data = data.rename(columns=data.iloc[2713723])[:2713723]
for col in ['order_date','close_date','issue_date','change_timestamp']:
    data[col] = pd.to_datetime(data[col], errors='coerce')

Wall time: 1min 4s


In [4]:
data.service_title.nunique()

1540

In [8]:
data.head(10)

,order_date,requester,service,cpgu_user,service_title,receipt_mfc,order_number,mfc,internal_status,external_status,sub_department,creation_mfc,order_type,department_id,deleted,deleter_fk,custom_service_id,close_date,service_level,issue_date,change_timestamp
0,2019-01-06 15:55:33.527,7184765,43,193686,134,6559747,5639097,6559747,10,1853152,NaN,6559747,SERVICE,316134,0,NaN,4.83333e+07,2019-01-06,FEDERAL,2019-01-06,2019-01-06 16:41:09.367
1,2019-01-06 16:42:30.203,7184765,46880280,193686,491,6559747,5639098,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,2019-01-06,FEDERAL,2019-01-06,2019-01-06 16:49:07.493
2,2019-01-06 17:40:47.163,4579720,43,193686,815,6559747,5639099,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298,2019-01-06,FEDERAL,2019-01-06,2019-01-06 17:42:28.270
3,2019-01-08 14:24:48.943,7184765,43,193686,134,6559747,5639100,6559747,10,1853152,NaN,6559747,SERVICE,316134,0,NaN,4.83333e+07,2019-01-08,FEDERAL,2019-01-08,2019-01-08 15:17:09.787
4,2019-01-08 15:19:51.693,7184765,46880280,193686,491,6559747,5639101,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,2019-01-08,FEDERAL,2019-01-08,2019-01-08 15:20:31.417
5,2019-01-08 15:28:17.503,4579720,43,193686,815,6559747,5639102,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298,2019-01-08,FEDERAL,2019-01-08,2019-01-08 15:29:53.653
6,2019-01-09 08:42:55.627,184297601,43,16732367,234,15224350,5639103,15224350,10,1853152,NaN,15224350,SERVICE,49801718,0,NaN,331384,2019-01-16,FEDERAL,2019-01-16,2019-01-16 17:09:04.337
7,2019-01-09 08:51:48.840,184297930,43,16732367,234,15224350,5639104,15224350,10,1853152,NaN,15224350,SERVICE,49801718,0,NaN,331384,2019-01-17,FEDERAL,2019-01-17,2019-01-17 16:59:09.297
8,2019-01-09 08:57:43.967,16760150,2266851,149837815,524,1788235,5639105,1788235,10,5,NaN,1788235,SERVICE,316160,0,NaN,NaN,2019-01-09,FEDERAL,2019-01-09,2019-01-09 08:57:58.920
9,2019-01-09 09:02:26.490,50274725,43,73486914,234,19136693,5639106,19136693,10,5,NaN,19136693,SERVICE,49801718,0,NaN,331384,2019-01-10,FEDERAL,NaT,2019-01-10 17:12:13.773


### No ML Modeling

Попробуем проверить качество предсказаний, основываясь на паттернах последовательности последних N услуг. Например, для тестовых клиентов с последними двумя услугами: [57,78], предскажем последнее значение наиболее подходящего паттерна; [57,78,43] , то есть 43.

In [ ]:
df = data.sort_values(by = ['requester','order_date'])
df['service2_title'] = strain.groupby('requester')['service_title'].shift(-1)
df['service3_title'] = strain.groupby('requester')['service_title'].shift(-2)
df['tuple'] = df[['service_title','service2_title']].apply(tuple,axis=1)
df['triple'] =df[['service_title','service2_title','service3_title']].apply(tuple,axis=1)
df2 = df[['requester','tuple','triple']]
df2['temp1'] = df['tuple'].astype(str) + df['requester'].astype(str)
df2['temp2'] = df['triple'].astype(str) + df['requester'].astype(str)

#st.drop_duplicates(['temp1'],keep = 'last')
from collections import Counter
d2 = dict(Counter([(int(x[0]),int(x[1])) for x in list(df2.drop_duplicates(['temp1'],keep = 'last')['tuple'].values) if sum(x)>0]).most_common())
d3 = dict(Counter([(int(x[0]),int(x[1]),int(x[2])) for x in list(df2.drop_duplicates(['temp2'],keep = 'last')['triple'].values) if sum(x)>0]).most_common())

In [ ]:
for s in sub['requester'].unique():
    services = data.loc[data['requester'] == s,'service_title'].values
    if len(services)>0:
        try:
            sub.loc[sub['requester'] == s,'Pairs'] = list({k:v for k,v in d2.items() if k[0] == services[-1]}.keys())[0][-1]
        except IndexError:
            pass
    if len(services)>1:
        try:
            sub.loc[sub['requester'] == s,'Triples'] = list({k:v for k,v in d3.items() if (k[0],k[1]) == (services[-2],services[-1])}.keys())[0][-1]
        except IndexError:
            pass  
    print(sub[sub['requester'] == s].values)

In [ ]:
sub['service_title'] = sub['Pairs']
#sub[['requester','service_title']].to_csv('sub2_17july.csv',index=False)

In [ ]:
sub.loc[sub['Triples'] !=1,'service_title'] = sub['Triples']
sub.loc[sub['Triples'] ==1,'service_title'] = sub['Pairs']
sub[['requester','service_title']].to_csv('sub3_17july.csv',index=False)

Получаем accuracy = 0,237. Также если копировать последнюю услугу из истории услуг клиента в предсказания, то accuracy на лидерборде будет 0,19. Для задачи с 1540 классами возможно это неплохой результат, но пойдём дальше; проведём пре-процессинг данных и обучим классификатор CatBoost.

### Pre-processing + ML

In [ ]:
requester = pd.read_csv('requester.csv',sep=';',engine='python')
requester.rename(columns={'requester_type':'retype','birth_year':'age','gender':'gen','edit_timestamp':'edit_file','deleted':'del'},inplace=True)
requester['age'] = 2020 - requester['age']
requester['edit_file'] = pd.to_datetime(requester['edit_file'], errors='coerce')
requester.loc[requester['gen'].isna(),'gen'] = 0
requester.loc[requester['age'].isna(),'age'] = 0
for col in ['retype','id','gen','del','age']:
    requester[col] = requester[col].astype(int)
    
data = data.merge(requester,left_on='requester',right_on = 'id',how='left')
del data['id']

In [ ]:
sub = pd.read_csv('sample_submission.csv')
print('Всего клиентов -',data['requester'].nunique(),', из которых тестовых -',sub.requester.nunique())
print('Всего обращений -',data.shape[0],', из которых обращений тестовых клиентов -', data[data['requester'].isin(sub.requester.unique())].shape[0])

### Добавление данных из остальных таблиц

In [ ]:
cpgu_service = pd.read_csv('cpgu_service.csv',sep=';', engine='python')
for col in ['is_not_render','person','sole','legal']:
    cpgu_service[col] = cpgu_service[col].astype(int)
dict_eid = dict(zip(['0', '103800001', '103800002', '103800003', '103800004', '103800005', '103800007', '103800008', '103800009', '103800010', '103800011', '103800013', '103800014', '103800015', '103800018', '103800020', '103800024', '103800025', '103800026', '103800027', '103800028', '103800029', '103800030', '103800031', '103800033', '103800034', '103800035', '103800036', '103800037', '103800040', '103800041', '103800042', '103800043', '103800044', '103800046', '103800047', '103800048', '103800049', '103800050', '103800052', '103800053', '103800054', '103800055', '103800057', '103800061', '103800062', '103800063', '103800064', '103800065', '103800067', '103800068', '103800069', '103800070', '103800071', '103800072', '103800077', '103800078', '103800079', '103800080', '103800081', '103800083', '103800084', '103800085', '103800086', '103800087', '103800088', '103800089', '103800090', '103800091', '103800092', '103800095', '103800096', '103800098', '103800102', '103800103', '103800104', '103800105', '103800106', '103800109', '103800110', '103800125', '103800127', '103800128', '103800134', '103800135', '103800145', '103800154', '103800156', '103800157', '103800158', '103800170', '103800171', '103800172', '103800175', '103800178', '103800182', '103800183', '103800184', '103800193', '103800196', '103800201', '103800202', '103800203', '103800220', '103800221', '103800225', '103800259', '103800260', '103800272', '103800273', '103800274', '103800291', '103800292', '103800293', '103800294', '103800313', '103800327', '103800337', '103800352', '103810313', 'basis_3800000010000299116', 'bazis_', 'bazis_3800000010000285185', 'bazis_3800000010000285641', 'bazis_3800000010000290028', 'bazis_3800000010000290324', 'bazis_3800000010000292885', 'bazis_3800000010000295987', 'bazis_3800000010000296228', 'bazis_3800000010000296463', 'bazis_3800000010000296643', 'bazis_3800000010000297996', 'bazis_3800000010000299116', 'bazis_3800000010000299917', 'bazis_3800000010000318739', 'bazis_3800000010000319184', 'bazis_3800000010000320268', 'bazis_3800000010000320851', 'bazis_3800000010000326446', 'bazis_3800000010000329775', 'election2018v2', 'election2018v3', 'election2018v4', 'esia/confirmUser', 'esia/findAccount', 'esia/getRequestResult', 'esia/recoverUser', 'esia/registerUser', 'fns/fnsZadorg', 'fss10001294259', 'fss10001437599', 'msp100000001', 'msp100005699', 'msp100005700', 'msp100005896', 'msp100005897', 'msp100005898', 'mvd/giacCriminalRecord', 'mvd/offenceDrugs', 'pfr/pfrRegistration', 'smartKit_3800000010000001631_3800000000003800700', 'test'],
                    ['a, a0', 'abc, 103800001', 'abc, 103800002', 'abc, 103800003', 'abc, 103800004', 'abc, 103800005', 'abc, 103800007', 'abc, 103800008', 'abc, 103800009', 'abc, 103800010', 'abc, 103800011', 'abc, 103800013', 'abc, 103800014', 'abc, 103800015', 'abc, 103800018', 'abc, 103800020', 'abc, 103800024', 'abc, 103800025', 'abc, 103800026', 'abc, 103800027', 'abc, 103800028', 'abc, 103800029', 'abc, 103800030', 'abc, 103800031', 'abc, 103800033', 'abc, 103800034', 'abc, 103800035', 'abc, 103800036', 'abc, 103800037', 'abc, 103800040', 'abc, 103800041', 'abc, 103800042', 'abc, 103800043', 'abc, 103800044', 'abc, 103800046', 'abc, 103800047', 'abc, 103800048', 'abc, 103800049', 'abc, 103800050', 'abc, 103800052', 'abc, 103800053', 'abc, 103800054', 'abc, 103800055', 'abc, 103800057', 'abc, 103800061', 'abc, 103800062', 'abc, 103800063', 'abc, 103800064', 'abc, 103800065', 'abc, 103800067', 'abc, 103800068', 'abc, 103800069', 'abc, 103800070', 'abc, 103800071', 'abc, 103800072', 'abc, 103800077', 'abc, 103800078', 'abc, 103800079', 'abc, 103800080', 'abc, 103800081', 'abc, 103800083', 'abc, 103800084', 'abc, 103800085', 'abc, 103800086', 'abc, 103800087', 'abc, 103800088', 'abc, 103800089', 'abc, 103800090', 'abc, 103800091', 'abc, 103800092', 'abc, 103800095', 'abc, 103800096', 'abc, 103800098', 'abc, 103800102', 'abc, 103800103', 'abc, 103800104', 'abc, 103800105', 'abc, 103800106', 'abc, 103800109', 'abc, 103800110', 'abc, 103800125', 'abc, 103800127', 'abc, 103800128', 'abc, 103800134', 'abc, 103800135', 'abc, 103800145', 'abc, 103800154', 'abc, 103800156', 'abc, 103800157', 'abc, 103800158', 'abc, 103800170', 'abc, 103800171', 'abc, 103800172', 'abc, 103800175', 'abc, 103800178', 'abc, 103800182', 'abc, 103800183', 'abc, 103800184', 'abc, 103800193', 'abc, 103800196', 'abc, 103800201', 'abc, 103800202', 'abc, 103800203', 'abc, 103800220', 'abc, 103800221', 'abc, 103800225', 'abc, 103800259', 'abc, 103800260', 'abc, 103800272', 'abc, 103800273', 'abc, 103800274', 'abc, 103800291', 'abc, 103800292', 'abc, 103800293', 'abc, 103800294', 'abc, 103800313', 'abc, 103800327', 'abc, 103800337', 'abc, 103800352', 'abc, 103810313', 'basis, 3800000010000299116', 'bazis, bazis0', 'bazis, 3800000010000285185', 'bazis, 3800000010000285641', 'bazis, 3800000010000290028', 'bazis, 3800000010000290324', 'bazis, 3800000010000292885', 'bazis, 3800000010000295987', 'bazis, 3800000010000296228', 'bazis, 3800000010000296463', 'bazis, 3800000010000296643', 'bazis, 3800000010000297996', 'bazis, 3800000010000299116', 'bazis, 3800000010000299917', 'bazis, 3800000010000318739', 'bazis, 3800000010000319184', 'bazis, 3800000010000320268', 'bazis, 3800000010000320851', 'bazis, 3800000010000326446', 'bazis, 3800000010000329775', 'election, 2018v2', 'election, 2018v3', 'election, 2018v4', 'esia, confirmUser', 'esia, findAccount', 'esia, getRequestResult', 'esia, recoverUser', 'esia, registerUser', 'fns, fnsZadorg', 'fss, 10001294259', 'fss, 10001437599', 'msp, 100000001', 'msp, 100005699', 'msp, 100005700', 'msp, 100005896', 'msp, 100005897', 'msp, 100005898', 'mvd, giacCriminalRecord', 'mvd, offenceDrugs', 'pfr, pfrRegistration', 'smartKit, 3800000010000001631_3800000000003800700', 'test, test0']))

cpgu_service['eid'] = cpgu_service['eid'].map(dict_eid)
cpgu_service['eid1'] = cpgu_service['eid'].astype(str).str.split(', ').str[0]
cpgu_service['eid2'] = cpgu_service['eid'].astype(str).str.split(', ').str[1]

data['service'] = data['service'].astype(int)
cpgu_service['id'] = cpgu_service['id'].astype(int)
data = data.merge(cpgu_service[['id','description','is_not_render','person','sole','legal','eid1','eid2','department']], left_on = 'service', right_on = 'id',how='left')
del data['id']
print('Всего в датасете категорий обращений',data['service'].nunique(),'из возможных - ',cpgu_service['id'].nunique())

In [ ]:
#добавим порядковый рабочий день от 1 до 433
days = data.sort_values(['order_date'])[['order_date']]
days['order_date'] = days['order_date'].astype(str).str.split(' ').str[0]

c=1
data['day'] = data['order_date'].astype(str).str.split(' ').str[0]
for n in days['order_date'].unique():
    data.loc[data['day']==n,'day'] = c
    c+=1

In [ ]:
#data.sort_values(['requester','order_date'],ascending=[True,True],inplace=True)

In [ ]:
#удалим битые строки из датасета:
r1 = pd.to_numeric(data['receipt_mfc'], errors='coerce')
data = data.iloc[r1[r1.notna()].index]

data.loc[data.index==1284367,'order_number'] = 6930981
for col in ['requester','service_title','receipt_mfc','mfc','order_number']: 
    data[col] = data[col].astype(int)

In [ ]:
data['weekday'] = data['order_date'].dt.weekday
data['month'] = data['order_date'].dt.month
data['week'] = data['order_date'].dt.week
data['day_month'] = data['order_date'].dt.day
data['year'] = data['order_date'].dt.year
data

In [ ]:
cpgu_mfc = pd.read_csv('cpgu_mfc.csv',sep=';', engine='python')
cpgu_mfc['id'] = cpgu_mfc['id'].astype(int)
cpgu_mfc.loc[cpgu_mfc['address'].str.startswith('Иркутская область, '),'address'] = cpgu_mfc.loc[cpgu_mfc['address'].str.startswith('Иркутская область, '),'address'].str[19:]
#train['address'] = 'Иркутская область, ' + train['address']
districts = dict(zip(list(cpgu_mfc['address'].unique()),['город, Иркутский район, Иркутск, Пискунова улица, д.160',
 'город, Тайшетский район, Бирюсинск, Горького улица, д.1/36',
 'поселок, Аларский район, Кутулик, Советская улица, д.50А',
 'город, Иркутский район, Иркутск, Рябикова бульвар, д.22А',
 'город, Усольский район, Усолье-Сибирское, Ленинский проспект, д.11/1',
 'город, Ангарский район, Ангарск, Ворошилова улица, д.65',
 'город, Слюдянский район, Байкальск, Южный микрорайон, 1-й квартал, д.26',
 'город, Зиминский район, Саянск, Строителей микрорайон, д.26',
 'город, Тайшетский район, Тайшет, Гагарина улица, д.115А',
 'село, Иркутский район, Хомутово, Колхозная улица, д.135',
 'поселок, Боханский район, Бохан, Колхозная улица, д.7',
 'село, Ольхонский район, Еланцы, Ленина улица, д.48',
 'город, Нижнеилимский район, Железногорск-Илимский, Янгеля улица, д.12',
 'город, Шелеховский район, Шелехов, 1-й квартал, д.10',
 'город, Братский район, Братск, Энергетик жилой район, Юбилейная улица, д.15',
 'город, Тулунский район, Тулун, Ленина улица, д.83',
 'город, Иркутский район, Иркутск, Байкальская улица, д.340/1',
 'город, Усть-Илимский район, Усть-Илимск, Мира проспект, д.9',
 'город, Иркутский район, Иркутск, Декабрьских Событий улица, д.117',
 'город, Нижнеудинский район, Нижнеудинск, Октябрьская улица, д.1-2',
 'рабочий поселок, Усть-Удинский район, Усть-Уда, 50 лет Октября улица, д.22А',
 'город, Иркутский район, Иркутск, Трактовая улица, д.35',
 'город, Братский район, Братск, Центральный жилой район, Ленина проспект, д.37',
 'город, Зиминский район, Зима, Клименко улица, д.37',
 'город, Иркутский район, Иркутск, Клары Цеткин улица, д.12/1',
 'рабочий поселок, Заларинский район, Залари, Гагарина улица, д.4',
 'город, Бодайбинский район, Бодайбо, Урицкого улица, д.15',
 'рабочий поселок, Нижнеилимский район, Рудногорск, Первомайская улица, д.6А',
 'город, Иркутский район, Иркутск, Верхняя Набережная улица, д.10',
 'город, Братский район, Вихоревка, Дзержинского улица, д.66Б',
 'город, Иркутский район, Иркутск, Юбилейный микрорайон, д.19/1',
 'город, Черемховский район, Черемхово, Некрасова улица, д.17',
 'город, Усть-Кутский район, Усть-Кут, Хорошилова улица, д.2А',
 'рабочий поселок, Качугский район, Качуг, Победы улица, д.6',
 'город, Иркутский район, Иркутск, Советская улица, д.58',
 'город, Слюдянский район, Слюдянка, Магистральная улица, д.2',
 'рабочий поселок, Куйтунский район, Куйтун, Красного Октября улица, д.18',
 'поселок, Эхирит-Булагатский район, Усть-Ордынский, Ленина улица, д.8',
 'рабочий поселок, Чунский район, Октябрьский, Октябрьская улица, д.39',
 'город, Братский район, Братск, Центральный жилой район, Баркова улица, д.43',
 'рабочий поселок, Тайшетский район, Юрты, Дружбы улица, д.6',
 'село, Баяндаевский район, Баяндай, Некунде улица, д.131',
 'поселок, Усть-Илимский район, Невон, Кеульская улица, д.9',
 'город, Черемховский район, Свирск, Молодежная улица, д.1А',
 'рабочий поселок, Казачинско-Ленский район, Магистральный, 17 Съезда ВЛКСМ улица, д.70',
 'город, Киренский район, Киренск, Центральный микрорайон, Красноармейская улица, д.2А',
 'рабочий поселок, Чунский район, Чунский, Свердлова улица, д.12',
 'рабочий поселок, Жигаловский район, Жигалово, Партизанская улица, д.71',
 'рабочий поселок, Тайшетский район, Новобирюсинский, Ленина улица, д.22',
 'село, Катангский район, Ербогачен, Чкалова улица, д.11',
 'рабочий поселок, Иркутский район, Маркова, Березовый микрорайон, д.75',
 'рабочий поселок, Казачинско-Ленский район, Улькан, Машурова улица, д.7',
 'рабочий поселок, Нижнеилимский район, Новая Игирма, 3-й квартал, д.31',
 'село, Братский район, Тангуй, Мира улица, д.20',
 'рабочий поселок, Чунский район, Лесогорск, Комсомольская улица, д.5',
 'поселок, Иркутский район, Дзержинск, Центральная улица, д.1А',
 'поселок, Ангарский район, Мегет, 1-й квартал, д.7',
 'село, Иркутский район, Пивовариха, Дачная улица, д.8',
 'рабочий поселок, Усольский район, Средний, Степная 3-я улица, д.1А',
 'рабочий поселок, Черемховский район, Михайловка,  квартал 1, д. № 5, стр. 1',
 'село, Черемховский район, Рысево, Российская улица, д.5',
 'рабочий поселок, Усть-Илимский район, Железнодорожный, улица Ленина, д.29',
 'село, Иркутский район, Оек, Кирова улица, д.91Д',
 'поселок, Нижнеилимский район, Березняки, Янгеля улица, д.25',
 'село, Эхирит-Булагатский район, Тугутуй, Степная улица, д.23',
 'село, Братский район, Большеокинское, Мира улица, д.40А',
 'село, Нижнеудинский район, Алыгджер, Советская улица, д.2',
 'поселок, Балаганский район, Балаганск, Кольцевая, д.61',
 'село, Черемховский район, Новогромово, Советская улица, д.15-1',
 'поселок, Нукутский район, Новонукутский, Хангалова улица, д.2А',
 'село, Куйтунский район, Карымск, Набережная улица, д.6',
 'город, Ангарский район, Ангарск, 84-й квартал, д.16',
 'село, Тайшетский район, Шелехово, Почтовая улица, д.1',
 'поселок, Усольский район, Железнодорожный, Комсомольская улица, д.28а',
 'село, Тайшетский район, Березовка, 40 лет Победы улица, д.19',
 'деревня, Иркутский район, Ревякина, Школьный переулок, д.3',
 'село, Нукутский район, Тангуты, ул. Нагорная, 7',
 'село, Зиминский район, Батама, Ленина улица, д.40',
 'город, Нижнеудинский район, Алзамай, Первомайская улица, д.119',
 'село, Осинский район, Усть-Алтан, Школьная улица, д.21',
 'поселок, Усть-Кутский район, Ния, Тбилисская улица, д.5',
 'село, Братский район, Калтук, Ленина улица, д.39Б',
 'село, Осинский район, Оса, Чапаева улица, д.2В/2',
 'село, Братский район, Покосное, Сибирская улица, д.16',
 'поселок, Братский район, Прибрежный, Школьный переулок, д.9',
 'рабочий поселок, Усольский район, Тельма, Крупской улица, д.11',
 'село, Тулунский район, Алгатуй, Солнечная улица, д.16',
 'поселок, Иркутский район, Молодежный, д.7',
 'поселок, Нижнеудинский район, Замзор, Рабочая улица, д.5',
 'рабочий поселок, Шелеховский район, Большой Луг, Степная улица, д.2А',
 'село, Усть-Удинский район, Молька, Радищева улица, д.25',
 'село, Тулунский район, Котик, Центральная улица, д.1А',
 'село, Боханский район, Хохорск, Ленина улица, д.44',
 'село, Усольский район, Сосновка, Лесная улица, д.1А',
 'рабочий поселок, Слюдянский район, Култук, Безымянный переулок, д.2',
 'село, Братский район, Кобляково, Наймушина улица, д.12',
 'село, Иркутский район, Смоленщина, Трудовая улица, д.12',
 'поселок, Мамско-Чуйский район, Мама, Октябрьская улица, д.23',
 'поселок, Тайшетский район, Соляная, Береговая улица, д.3',
 'рабочий поселок, Заларинский район, Тыреть 1-я, Солерудник микрорайон, д.9',
 'рабочий поселок, Усольский район, Мишелевка, Титова улица, д.1А',
 'деревня, Иркутский район, Карлук, Гагарина улица, д.4Б',
 'рабочий поселок, Усольский район, Белореченский, Вторая улица, д.100В',
 'рабочий поселок, Тайшетский район, Квиток, Октябрьская улица, д.10',
 'рабочий поселок, Нижнеилимский район, Видим, Нагорная улица, д.1А',
 'село, Тулунский район, Будагово, Заводская улица, д.8А',
 'село, Ангарский район, Савватеевка, Школьная улица, д.48',
 'рабочий поселок, Нижнеудинский район, Шумский, Заозерная улица, д.2',
 'село, Тулунский район, Гуран, Бурлова улица, д.36',
 'село, Заларинский район, Моисеевка, Школьная улица, д.2А',
 'деревня, Иркутский район, Ширяева, Специалистов улица, д.1',
 'село, Усольский район, Новожилкино, Совхозная 1-я улица, д.13',
 'село, Зиминский район, Кимильтей, Чкалова улица, д.70А',
 'село, Братский район, Ключи-Булак, Ленина улица, д.1',
 'село, Куйтунский район, Каразей, Мира улица, д.58',
 'поселок, Нижнеилимский район, Речушка, Пионерская улица, д.19',
 'село, Черемховский район, Голуметь, Калинина улица, д.10',
 'село, Иркутский район, Урик, Ленина улица, д.1',
 'поселок, Зиминский район, Центральный Хазан, Мира улица, д.57',
 'село, Черемховский район, Нижняя Иреть, Советская улица, д.37А',
 'село, Боханский район, Казачье, Мира улица, д.10',
 'село, Эхирит-Булагатский район, Гаханы, Гагарина улица, д.6',
 'село, Усольский район, Большая Елань, Победы улица, д.2',
 'село, Балаганский район, Кумарейка, Первомайская улица, д.2',
 'рабочий поселок, Тайшетский район, Шиткино, Кирова улица, д.26',
 'село, Баяндаевский район, Хогот, Трактовая улица, д.65',
 'село, Тайшетский район, Джогино, Больничная улица, д.8',
 'рабочий поселок, Усольский район, Тайтурка, Пеньковского улица, д.8',
 'село, Качугский район, Манзурка, Трактовая улица, д.76',
 'село, Иркутский район, Никольск, Черемуховая улица, д.1а',
 'село, Черемховский район, Парфеново, Мира улица, д.25',
 'село, Аларский район, Аларь, Советская улица, д.43',
 'село, Боханский район, Каменка, Школьная улица, д.8',
 'рабочий поселок, Нижнеудинский район, Атагай, Победы улица, д.4',
 'поселок, Куйтунский район, Тулюшка, Мира улица, д.11',
 'село, Иркутский район, Малое Голоустное, Мира улица, д.25А',
 'рабочий поселок, Усть-Кутский район, Янталь, Еловая улица, д.13',
 'поселок, Осинский район, Приморский, Гагарина улица, д.25А',
 'село, Казачинско-Ленский район, Казачинское, Советская улица, д.4',
 'поселок, Чунский район, Новочунка, ул. Толстого, д.15',
 'село, Усть-Удинский район, Новая Уда, Юбилейная улица, д.1',
 'село, Качугский район, Харбатово, ул. Совхозная, 14',
 'поселок, Нукутский район, Новоленино, Школьный переулок, д.6',
 'село, Шелеховский район, Баклаши, Ангарская улица, д.44',
 'рабочий поселок, Нижнеилимский район, Шестаково, Ленина улица, д.20А',
 'деревня, Иркутский район, Усть-Куда, Геологическая улица, д.4',
 'село, Заларинский район, Бажир, Юбилейная улица, д.14',
 'поселок, Тулунский район, 4 отделение Государственной селекционной станции, ул. Мичурина, 36',
 'село, Черемховский район, Зерновое, Иркутская улица, д.10',
 'село, Черемховский район, Алехино, Полевая улица, д.4',
 'поселок, Братский район, Кежемский, Первомайская улица, д.6',
 'село, Боханский район, Олонки, Калинина улица, д.5',
 'село, Аларский район, Аляты, Ж.Зимина улица, д.1',
 'село, Заларинский район, Троицк, Молодежная улица, д.4А',
 'село, Куйтунский район, Уян, 1-я Советская улица, д.6',
 'село, Аларский район, Иваническое, Юбилейная улица, д.7',
 'село, Иркутский район, Горохово, Школьная улица, д.15',
 'рабочий поселок, Нижнеилимский район, Радищев, Первая улица, д.2',
 'село, Иркутский район, Максимовщина, Сибирская улица, д.16а',
 'село, Осинский район, Ново-Ленино, Ленина улица, д.1',
 'поселок, Нижнеудинский район, Костино, Новая улица, д.31',
 'поселок, Усть-Илимский район, Тубинский, Таежная улица, д.5',
 'поселок, Ольхонский район, Бугульдейка, Больничный переулок, д.7',
 'поселок, Братский район, Турма, Строительная улица, д.12',
 'село, Тайшетский район, Николаевка, Первомайская улица, д.14',
 'поселок, Эхирит-Булагатский район, Свердлово, Советская улица, д.19',
 'поселок, Аларский район, Забитуй, 70 лет Октября улица, д.24',
 'поселок, Усть-Кутский район, Верхнемарково, 40 лет Победы улица, д.47',
 'село, Ангарский район, Одинск, Победы улица, д.7',
 'деревня, Шелеховский район, Олха, Советская улица, д.21Б',
 'село, Тулунский район, Икей, Коммуны улица, д.126',
 'село, Нижнеудинский район, Шеберта, Трактовая улица, д.2',
 'село, Куйтунский район, Барлук, Ленина улица, д.29А',
 'село, Нижнеудинский район, Худоеланское, Московская улица, д.45А',
 'село, Тулунский район, Бадар, Перфиловская улица, д.1',
 'поселок, Усть-Илимский район, Эдучанка, Ермака улица, д.4',
 'село, Осинский район, Бильчир, Ленина улица, д.24',
 'поселок, Усольский район, Новомальтинск, 2-й квартал, д.1',
 'рабочий поселок, Иркутский район, Листвянка, Горького улица, д.89',
 'рабочий поселок, Нижнеилимский район, Янгель, Космонавтов микрорайон, д.9А',
 'село, Нукутский район, Хадахан, Административная улица, д.2',
 'город, Иркутский район, Иркутск, 5-й Армии улица, д.2/1',
 'деревня, Баяндаевский район, Загатуй, 1-й микрорайон, д.41',
 'село, Усольский район, Мальта, ул. Школьная, 21 А',
 'поселок, Усть-Илимский район, Седаново, Кирова улица, д.33',
 'рабочий поселок, Нижнеилимский район, Хребтовая, Калинина улица, д.1',
 'рабочий поселок, Нижнеудинский район, Ук, Кимильтейская улица, д.3А',
 'село, Боханский район, Тихоновка, Ленина улица, д.13',
 'село, Тулунский район, Перфилово, 50 лет Октября улица, д.39',
 'рабочий поселок, Иркутский район, Большая Речка, Труда улица, д.28',
 'поселок, Усольский район, Раздолье, Мира улица, д.27',
 'деревня, Иркутский район, Сосновый Бор, Урожайная улица, д.14',
 'поселок, Тайшетский район, Тамтачет, Гайнулина улица, д.1А',
 'деревня, Тулунский район, Афанасьева, Ленина улица, д.4А',
 'поселок, Усть-Кутский район, Ручей, Школьная улица, д.3',
 'село, Нижнеудинский район, Катарбей, Советская улица, д.84',
 'деревня, Эхирит-Булагатский район, Нижняя Идыга, ул. Шабагановская, д. 2',
 'село, Тулунский район, Шерагул, Ленина улица, д.84',
 'поселок, Усольский район, Тальяны, Клубная улица, д.2',
 'село, Черемховский район, Бельск, Иванова улица, д.56',
 'город, Усольский район, Усолье-Сибирское, Орджоникидзе улица, д.31а',
 'село, Зиминский район, Самара, ул. Черемушки, д. 2',
 'село, Тулунский район, Гадалей, Сорокина улица, д.54',
 'село, Осинский район, Обуса, 11 Комсомольцев улица, д.6',
 'город, Иркутский район, Иркутск, пр-т Большой Литейный, 3',
 'поселок, Ольхонский район, Хужир, Байкальская улица, д.12',
 'рабочий поселок, Киренский район, Алексеевск, Чапаева улица, д.65',
 'село, Нижнеудинский район, Верхняя Гутара, Центральная улица, д.11',
 'село, Эхирит-Булагатский район, Харат, Ленина, д.24',
 'село, Тулунский район, Едагон, Ленина, д.66',
 'город, Братский район, Братск, Гидростроитель жилой район, Вокзальная улица, д.2А',
 'город, Иркутский район, Иркутск, Рабочая улица, д.2А']))

cpgu_mfc['address_norm'] = cpgu_mfc['address'].map(districts)

#train['a1'] = train['address_norm'].astype(str).str.split(", ").str[0]
#train['a2'] = train['address_norm'].astype(str).str.split(", ").str[1]
#train['a3'] = train['address_norm'].astype(str).str.split(", ").str[2]

In [ ]:
cpgu_mfc.drop(columns = ['name','full_name','address','key_mfc','damask_office_id','code'],axis =1,inplace=True)
temp = cpgu_mfc[['id','address_norm']]
temp.columns = ['ids','address_parent']
cpgu_mfc = cpgu_mfc.merge(temp,left_on='parent_id',right_on='ids',how='left')

In [ ]:
del cpgu_mfc['ids']
data = data.merge(cpgu_mfc,left_on='receipt_mfc',right_on = 'id',how='left')
del data['id']
gc.collect()

In [ ]:
data['office_type_id'] = np.abs(data['office_type_id'] - 2) #???
data['office_type_id'] = data['office_type_id'].astype(int)
data.rename(columns={'office_type_id':'office'},inplace=True)
data

In [ ]:
cpgu_department = pd.read_csv('cpgu_department.csv',sep=';', engine='python')
temp = cpgu_department[['id','title']]
temp.columns = ['ids','parent_title']
cpgu_department = cpgu_department.merge(temp,left_on='parent',right_on='ids',how='left')
cpgu_department = cpgu_department[['id','title','parent_title']]
cpgu_department.columns = ['id','d1','d2']
data = data.merge(cpgu_department,left_on='department_id',right_on = 'id',how='left')
del data['id']

In [ ]:
cpgu_user = pd.read_csv('cpgu_user.csv',sep=';', engine='python')
cpgu_user['auto_ping_queue'] = cpgu_user['auto_ping_queue'].astype(int)
cpgu_user.columns = ['cpgu_user_id','autouser']
data = data.merge(cpgu_user,left_on='cpgu_user',right_on = 'cpgu_user_id',how='left')
gc.collect()

In [ ]:
del data['cpgu_user_id']

### custom_service_info

In [ ]:
data.loc[data['custom_service_id'].notna(),'custom_service_id'] = data.loc[data['custom_service_id'].notna(),'custom_service_id'].astype(int)

In [ ]:
custom_service_info = pd.read_csv('custom_service_info.csv',sep=';', engine='python')
custom_service_info = custom_service_info[['id','title','group_title']]
custom_service_info.columns = ['ids','custom_title','custom_group']
data = data.merge(custom_service_info,left_on='custom_service_id',right_on = 'ids',how='left')
del data['ids']

In [ ]:
data['uno_address'] = 0
data.loc[data['address_norm']==data['address_norm'],'uno_address'] = 1

In [ ]:
office_type = pd.read_csv('office_type.csv',sep=';', error_bad_lines=False,engine='python')
office_type.head(2)

In [ ]:
order_relates = pd.read_csv('order_relates.csv',sep=';', error_bad_lines=False,engine='python')
order_relates.info()

In [ ]:
gc.collect()

In [ ]:
order_history_agg = pd.read_pickle('order_history_agg')
data = data.merge(order_history_agg,left_on='order_number',right_on='cpgu_order',how='left')
del data['cpgu_order']

In [ ]:
data['a1'] = data['address_norm'].astype(str).str.split(", ").str[0]
data['a2'] = data['address_norm'].astype(str).str.split(", ").str[1]
data['a3'] = data['address_norm'].astype(str).str.split(", ").str[2]
data['p1'] = data['address_parent'].astype(str).str.split(", ").str[0]
data['p2'] = data['address_parent'].astype(str).str.split(", ").str[1]
data['p3'] = data['address_parent'].astype(str).str.split(", ").str[2]

In [ ]:
data['edit_file'] = (data['edit_file'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
data['edit_file'] = data['edit_file'] - data['edit_file'].min()

data['time_order'] = (data['order_date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
data['time_order'] = data['time_order'] - data['time_order'].min()

data['start_time'] = (data['order_date'] - data['order_date'].dt.normalize()) / pd.Timedelta('1 second')
data['start_time'] = data['start_time']/3600

data['end_time'] = (data['change_timestamp'] - data['change_timestamp'].dt.normalize()) / pd.Timedelta('1 second')
data['end_time'] = data['end_time']/3600

data['time_order_change'] = (data['change_timestamp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
data['time_order_change'] = data['time_order_change'] - data['time_order_change'].min()

data['delta_time'] = data['end_time'] - data['start_time']

In [ ]:
data.fillna(inplace=True)
#data.to_pickle('final_data_1')

In [ ]:
data.drop(['order_date','close_date','issue_date','issue_date','change_timestamp'],axis=1,inplace=True)
data.sort_values(['requester','time_order'],ascending=[True,True],inplace=True)

In [ ]:
data.loc[~data['requester'].isin(sub['requester'].unique()),'rank'] = data[~data['requester'].isin(sub['requester'].unique())].groupby(['requester','day']).cumcount()
data.loc[data['requester'].isin(sub['requester'].unique()),'rank'] = data[data['requester'].isin(sub['requester'].unique())].groupby(['requester','day']).cumcount()+1
data['rank'] = data['rank'].astype(int)

In [ ]:
equeue_ticket_order_act = pd.read_pickle('equeue_ticket_order_act')
equeue_ticket = pd.read_pickle('equeue_ticket')
equeue_ticket['seconds'] = (equeue_ticket.serving_finished - equeue_ticket.serving_started).dt.total_seconds()
equeue_ticket_order_act = pd.read_pickle('equeue_ticket_order_act')
equeue_ticket_order_act = equeue_ticket_order_act.merge(equeue_ticket,left_on = 'equeue_ticket',right_on='id', how='left')
del equeue_ticket

data = data.merge(equeue_ticket_order_act[['cpgu_order','service_name','seconds']].drop_duplicates(['cpgu_order','service_name']),left_on='order_number',right_on='cpgu_order',how='left')
del equeue_ticket_order_act

In [ ]:
equeue_ticket_order_act.sort_values(['cpgu_order','serving_started'],ascending=[True,False],inplace=True)
equeue_ticket_order_act['size'] = equeue_ticket_order_act['cpgu_order'].map(equeue_ticket_order_act['cpgu_order'].value_counts())
equeue_ticket_order_act

In [ ]:
equeue_ticket_order_act.sort_values(['cpgu_order','serving_started'],ascending=[True,False],inplace=True)
equeue_ticket_order_act['size'] = equeue_ticket_order_act['cpgu_order'].map(equeue_ticket_order_act['cpgu_order'].value_counts())


In [ ]:
uslugi = equeue_ticket_order_act.service_name.value_counts(dropna=False).to_frame().reset_index()
uslugi2 = pd.read_excel('uslugi.xlsx')
dicto = dict(zip(uslugi['index'],uslugi2['index']))
equeue_ticket_order_act['service_name'] = equeue_ticket_order_act['service_name'].map(dicto)

equeue_ticket_order_act.loc[equeue_ticket_order_act['service_name']=='Выдача','service_name'] = 'Выдача документов'
equeue_ticket_order_act.loc[equeue_ticket_order_act['service_name']=='Выдача паспортов и водительских удостоверений','service_name'] = 'Выдача паспортов, водительских удостоверений, загранпаспортов старого образца (на 5 лет)'
equeue_ticket_order_act.loc[equeue_ticket_order_act['service_name']=='Выписка из Единого государственного реестра налогоплательщиков','service_name'] = 'Выписка из ЕГРН'
equeue_ticket_order_act.loc[equeue_ticket_order_act['service_name']=='Действия с учетной записью для портала Госуслуг','service_name'] = 'Портал Госуслуги'
equeue_ticket_order_act.loc[equeue_ticket_order_act['service_name']=='Изготовление водительского удостоверения','service_name'] = 'Водительское удостоверение'
equeue_ticket_order_act.loc[equeue_ticket_order_act['service_name']=='С Выдача платежных документов на уплату задолженности по налогам ФЛ','service_name'] = 'Выдача платежных документов на уплату задолженности по налогам ФЛ'

In [ ]:
equeue = equeue_ticket_order_act[['cpgu_order','type','serving_started','serving_finished','seconds','size','service_name']]
equeue.to_pickle('equeue')

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in ['service_name']:
    print(col)
    equeue[col] = le.fit_transform(equeue[col].astype(str))

In [ ]:
unique_orders = equeue[['cpgu_order','size']].drop_duplicates('cpgu_order',keep='first')

In [ ]:
equeue_last = equeue[['cpgu_order','service_name']].drop_duplicates(['cpgu_order'],keep='first')
equeue_first = equeue[['cpgu_order','service_name']].drop_duplicates(['cpgu_order'],keep='last')

equeue_last.columns = ['cpgu_order','last']
equeue_first.columns = ['cpgu_order','first']

In [ ]:
unique_orders = unique_orders.merge(equeue_first,on = 'cpgu_order',how='left')
unique_orders = unique_orders.merge(equeue_last,on = 'cpgu_order',how='left')
unique_orders.shape, unique_orders[unique_orders['size']>1].shape, unique_orders[unique_orders['size']>2].shape

In [ ]:
unique_orders.loc[unique_orders['size']>1,'status'] = unique_orders['first'].astype(str) + "_" + unique_orders['last'].astype(str)
unique_orders.loc[unique_orders['size']==1,'status'] = unique_orders['first'].astype(str) + "_" + 'size1'
unique_orders

In [ ]:
data = data.merge(unique_orders,left_on='order_number',right_on='cpgu_order',how='left')

In [ ]:
#data.to_pickle('final_data_3')

In [ ]:
cols = ['service','service_title']


In [ ]:
uslugi = equeue_ticket_order_act.service_name.value_counts(dropna=False).to_frame().reset_index()
uslugi2 = pd.read_excel('uslugi.xlsx')
dicto = dict(zip(uslugi['index'],uslugi2['index']))
equeue_ticket_order_act['service_name'] = equeue_ticket_order_act['service_name'].map(dicto)

equeue_ticket_order_act.loc[equeue_ticket_order_act['service_name']=='Выдача','service_name'] = 'Выдача документов'
equeue_ticket_order_act.loc[equeue_ticket_order_act['service_name']=='Выдача паспортов и водительских удостоверений','service_name'] = 'Выдача паспортов, водительских удостоверений, загранпаспортов старого образца (на 5 лет)'
equeue_ticket_order_act.loc[equeue_ticket_order_act['service_name']=='Выписка из Единого государственного реестра налогоплательщиков','service_name'] = 'Выписка из ЕГРН'
equeue_ticket_order_act.loc[equeue_ticket_order_act['service_name']=='Действия с учетной записью для портала Госуслуг','service_name'] = 'Портал Госуслуги'
equeue_ticket_order_act.loc[equeue_ticket_order_act['service_name']=='Изготовление водительского удостоверения','service_name'] = 'Водительское удостоверение'
equeue_ticket_order_act.loc[equeue_ticket_order_act['service_name']=='С Выдача платежных документов на уплату задолженности по налогам ФЛ','service_name'] = 'Выдача платежных документов на уплату задолженности по налогам ФЛ'

In [ ]:
equeue = equeue_ticket_order_act[['cpgu_order','type','serving_started','serving_finished','seconds','size','service_name']]
equeue.to_pickle('equeue')

In [ ]:
sub = pd.read_csv('sample_submission.csv')
train = data[~data['requester'].isin(sub['requester'].unique())]
train['size'] = train.requester.map(train.requester.value_counts())

train[train['size']<2].shape

data = data[~data.index.isin(train[train['size']<2].index)]

test_services=data.loc[data['requester'].isin(sub['requester'].unique()),'service_title'].unique()

data = data[data.service_title.isin(test_services)]

#data.to_pickle('final_data_2')

In [ ]:
for col in ['cons','deliv','serv']:
    
    data.loc[data[col]>1,col] = 1

In [ ]:
data['count_orders'] = data['cpgu_order'].map(data['cpgu_order'].value_counts())

In [ ]:
text = pd.read_excel('services_text.xlsx')
text.columns = ['ind','original_text','service_title','combo_title']
text.sample(10)

In [ ]:
data = data.merge(text[['service_title','combo_title']],on='service_title',how='left')
del data['target'], data['cpgu_order']

In [ ]:
data1 = pd.DataFrame()
cols = ['time_order','requester','order_number', 'service_title', 'service', 'combo_title', 'status', 
        'order_type', 'description', 'service_level','cpgu_user', 'receipt_mfc', 'mfc', 'internal_status', 
        'external_status', 'sub_department', 'creation_mfc', 'department_id', 'deleted', 'deleter_fk', 
        'custom_service_id', 'retype', 'gen', 'edit_file', 'del','age', 'description', 'is_not_render', 
        'person', 'sole', 'legal', 'eid1', 'eid2', 'department', 'day', 'weekday', 'month', 'week', 
        'day_month', 'year','closed', 'branch_id', 'win_count', 'qms_server_id', 'deperatment_id', 
        'office', 'parent_id', 'calendar', 'address_norm', 'address_parent', 'd1', 'd2', 'autouser', 
        'custom_title', 'custom_group', 'uno_address', 'in_0', 'in_1', 'in_2', 'in_3', 'in_4', 'in_5', 
        'in_6', 'in_9', 'in_10', 'in_1853149', 'ex_0', 'ex_1', 'ex_2', 'ex_4', 'ex_5', 'ex_6', 'ex_8', 
        'ex_9', 'ex_12', 'ex_15', 'ex_19', 'ex_1853152', 'ex_1853153', 'ex_1853154', 'ex_25230324', 'life', 
        'cons', 'deliv', 'serv', 's_serv', 's_cons', 's_deliv', 'a1', 'a2', 'a3', 'p1', 'p2', 'p3', 
        'start_time', 'end_time', 'time_order_change', 'delta_time', 'rank', 'size', 'first', 'last']
for col in cols:
    print(col)
    data1[col] = data[col]

In [ ]:
data1.columns = ['time','requester','order', 'service_title', 'service', 'combo', 'status', 'type', 'descr', 'level',
              'user', 'receipt_mfc', 'mfc', 'internal', 'external', 'sub', 
 'creation_mfc', 'depart_id', 'deleted', 'del_fk', 'custom_service', 'retype', 'gen', 'edit', 'del',
 'age', 'is_not_render', 'person', 'sole', 'legal', 'eid1', 'eid2', 'department', 'day', 'weekday', 'month', 'week', 'day_month', 'year',
 'closed', 'branch_id', 'win_count', 'qms_server_id', 'dep_id', 'office', 'parent', 'calendar', 'address', 'address_parent', 'd1', 'd2', 
 'autouser', 'custom_title', 'custom_group', 'uno_address', 'in_0', 'in_1', 'in_2', 'in_3', 'in_4', 'in_5', 'in_6', 'in_9', 'in_10', 'in_1853149', 'ex_0', 
 'ex_1', 'ex_2', 'ex_4', 'ex_5', 'ex_6', 'ex_8', 'ex_9', 'ex_12', 'ex_15', 'ex_19', 'ex_1853152', 'ex_1853153', 'ex_1853154', 'ex_25230324', 'life', 'cons', 
 'deliv', 'serv', 's_serv', 's_cons', 's_deliv', 'a1', 'a2', 'a3', 'p1', 'p2', 'p3', 'start_time', 'end_time', 'time_order_change', 
 'delta_time', 'rank', 'size', 'first', 'last']

data = data1
del data1
gc.collect()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in ['service','combo','type','descr','level','receipt_mfc','mfc','internal','external','sub','eid1','eid2',
            'dep_id','parent','calendar','address','address_parent','d1','d2','custom_title','a1','a2','a3','p1',
            'p2','p3','department','branch_id']:
    print(col)
    data[col] = le.fit_transform(data[col].astype(str))
    data[col] = data[col].astype(int)

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

reduce_mem_usage(data)

In [ ]:
data['service_title'] = data['service_title'].astype(str) + "_" + data['service'].astype(str)
data['service_title'].nunique()

In [ ]:
b = data['combo'].value_counts().index[250:]
data.loc[data['combo'].isin(b),'combo'] = 2000

stats = pd.get_dummies(data[['requester','combo']],columns=['combo'])
stats = stats.groupby('requester').agg('sum')
stats = stats.reset_index()
stats2 = stats.div(stats.sum(axis=1), axis=0)
stats = stats.merge(stats2,on='requester',how='left')
stats

In [ ]:
#data.to_pickle('final_data_4')

In [ ]:
data['time_for_same_action'] = data['time'] - data.groupby(['requester','service_title'])['time'].shift(-1)
data['time_for_same_combo'] = data['time'] - data.groupby(['requester','combo'])['time'].shift(-1)

In [ ]:
#data = pd.read_pickle('data_final')

for col in ['time', 'service', 'combo', 'type','coded_title','time_for_same_action','time_for_same_combo','first','last','start_time','address']:
    data["{}_past".format(col)] = data.groupby(['requester'])[col].shift(-1)
    data["{}_past2".format(col)] = data.groupby(['requester'])[col].shift(-2)
    data["{}_past3".format(col)] = data.groupby(['requester'])[col].shift(-3)
    #data["{}_past4".format(col)] = data.groupby(['requester'])[col].shift(-4)
    #data["{}_past5".format(col)] = data.groupby(['requester'])[col].shift(-5)
    
data = data.fillna(0)

In [ ]:
r=10
for df in [data]:
    df['t'] = ""
    for s in range(r):a
        if s == 0:
            df['t'] = df['t'].astype(str) + df['coded_title'].astype(str)+"_"+df['first'].astype(str)+"_"+df['last'].astype(str)
        if s > 0:
            df['t'] = df['t'].astype(str) + df.groupby('requester')['coded_title'].shift(-s).astype(str)+"_"+df.groupby('requester')['first'].shift(-s).astype(str)+"_"+df.groupby('requester')['last'].shift(-s).astype(str)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in ['t']:
    print(col)
    data[col] = le.fit_transform(data[col].astype(str))
    data[col] = data[col].astype(int)

In [ ]:
test = data[data['requester'].isin(sub['requester'].unique())].drop_duplicates('requester',keep='first').reset_index(drop=True)

In [ ]:
data['target'] = data.groupby('requester')['coded_title'].shift(+1)
data = data[data.target.notna()]
data = data[data['coded_title'].isin(test['coded_title'].unique())]
data = data[data['coded_title_past'].isin(test['coded_title_past'].unique())]
data = data[data['coded_title_past2'].isin(test['coded_title_past2'].unique())]
data = data[data['coded_title_past3'].isin(test['coded_title_past3'].unique())]
#data = data[data['coded_title_past4'].isin(test['coded_title_past4'].unique())]
#data = data[data['coded_title_past5'].isin(test['coded_title_past5'].unique())]

In [ ]:
min_date = data.groupby('requester')['time'].min().reset_index()
min_date.columns = ['requester','min_date']
data = data.merge(min_date,on='requester',how='left')

In [ ]:
data['target'] = data['target'].astype(int)

coded = pd.read_pickle('coded_title')
coded = coded.drop_duplicates(['service_title','coded_title'],keep='first')
#coded['service_title'] = coded['service_title'].astype(str).str.split(" ").str[0].astype(int)
dicto = dict(zip(coded['coded_title'],coded['service_title']))
#test['target'] = test['target'].map(dicto)
sub = pd.read_csv('sub1_24july.csv')

In [ ]:
data['count_target'] = data['target'].map(data['target'].value_counts())
data = data[data['count_target']>1].reset_index(drop=True)
del data['count_target']

In [ ]:
data = data[data['t'].isin(test['t'].unique())]
data['t'].nunique(),test['t'].nunique()

In [ ]:
cats = ['coded_title','combo','descr', 'user', 'receipt_mfc', 'mfc', 'internal', 'external', 
        'sub', 'creation_mfc', 'depart_id', 'custom_service', 'eid1', 'eid2', 'department', 'branch_id', 
        'qms_server_id', 'dep_id', 'parent', 'calendar', 'address', 'd1', 'd2', 'custom_title', 
        'custom_group', 'a1', 'a2', 'a3', 'p1', 'p2', 'p3','first', 'last', 
        
        'combo_past', 'type_past', 'coded_title_past', 'first_past', 
        'last_past', 'coded_title_past',
        'combo_past2', 'type_past2', 'coded_title_past2', 'first_past2', 
        'last_past2', 
        'coded_title_past2','combo_past3', 'type_past3', 'coded_title_past3', 'first_past3', 
        'last_past3', 
        'coded_title_past3']
['combo_past4', 'type_past4', 'coded_title_past4', 'first_past4', 
        'last_past4', 
        'coded_title_past4','combo_past5', 'type_past5', 'coded_title_past5', 'first_past5', 
        'last_past5', 
        'coded_title_past5'
       
       ]


for col in cats:
    data[col] = data[col].astype(int)
    test[col] = test[col].astype(int)
    
    
ids = [index for index, value in enumerate(data.columns) if value in cats]

del data['t'], test['t']


gc.collect()

In [ ]:
reduce_mem_usage(data)
reduce_mem_usage(test)

In [ ]:
print(data.shape,data.target.nunique())
gc.collect()

data['count_target'] = data['target'].map(data['target'].value_counts())
data = data[data['count_target']>1].reset_index(drop=True)
target = data['target']
del data['count_target'],data['target']
gc.collect()

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(data, target, test_size=0.5, random_state=0, stratify=target)
train_pool = Pool(data=train_X, label=train_y, cat_features=ids)
test_pool = Pool(data=test_X, label=test_y, cat_features=ids)
model = CatBoostClassifier(
    iterations = 2000,
    learning_rate=0.07,
    random_strength=0.1,
    depth=5,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    early_stopping_rounds = 100,verbose = 100)

model.fit(train_pool,plot=True, eval_set=test_pool)
test['target'] = model.predict(test.drop('target',axis=1))

In [ ]:
#обучение на каждой категории выбранного кат. признака на двух фолдах:
for category in data['category'].value_counts().index:
    train = data[data['category']=category]
    print(train.shape,train.target.nunique())
    gc.collect()

    train['count_target'] = train['target'].map(train['target'].value_counts())
    train = train[train['count_target']>1].reset_index(drop=True)
    target = train['target']
    del train['count_target'],train['target']


    train_X, test_X, train_y, test_y = train_test_split(train, target, test_size=0.5, random_state=0, stratify=target)
    train_pool = Pool(data=train_X, label=train_y, cat_features=ids)
    test_pool = Pool(data=test_X, label=test_y, cat_features=ids)
    model = CatBoostClassifier(
        task_type = 'GPU',
        iterations = 2000,
        learning_rate=0.07,
        random_strength=0.1,
        depth=5,
        loss_function='MultiClass',
        eval_metric='Accuracy',
        leaf_estimation_method='Newton',
        early_stopping_rounds = 100,verbose = 100)

    model.fit(train_pool,plot=False, eval_set=test_pool)
    te = test.loc[test['category']=category]
    test.loc[test['category']=category,'target'] = model.predict(te.drop('target',axis=1))